In [242]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import TensorDataset, DataLoader




In [243]:

class MLP(object):
    def __init__(self, numInput, numHidden, numOutput):
        self.fitness = 0
        self.numInput = numInput + 1 # Add bias node to inputs
        self.numHidden = numHidden
        self.numOutput = numOutput

        self.wh = np.random.randn(self.numHidden, self.numInput)
        self.wo = np.random.randn(self.numOutput, self.numHidden)

        self.ReLU = lambda x : max(0,x)

    def sigmoid(self,x):
        try:
            ans = (1 / (1 + math.exp(-x)))
        except OverflowError:
            ans = float('inf')
        return ans
    
    def feedForward(self, inputs):
      inputsBias = inputs[:]
      inputsBias.insert(len(inputs),1)                 # Add bias input
      h1 = np.dot(self.wh, inputsBias)                 # feed to hidden layer
      h1 = [self.ReLU(x) for x in h1]              # Activate hidden layer
      output = np.dot(self.wo, h1)                 # feed to output layer
      # output = [self.sigmoid(x) for x in output]   # Activate output layer
      return output
    
    def getWeightsLinear(self):
        flat_wh = list(self.wh.flatten())
        flat_wo = list(self.wo.flatten())
        return( flat_wh + flat_wo )

    def setWeightsLinear(self, Wgenome):
        numWeights_IH = self.numHidden * (self.numInput)
        self.wh = np.array(Wgenome[:numWeights_IH])
        self.wh = self.wh.reshape((self.numHidden, self.numInput))
        self.wo = np.array(Wgenome[numWeights_IH:])
        self.wo = self.wo.reshape((self.numOutput, self.numHidden))

myNet = MLP(numInputNodes, numHiddenNodes1, numOutputNodes)

In [244]:


# Dummy data for training and testing

all_data = pd.read_csv("eScooterDemand.csv")
all_data.head()



all_data['Date'] = pd.to_datetime(all_data['Date'], format='%d/%m/%Y')

## Prepare data for training
all_data['Windspeed'] = all_data['Wind speed']
all_data['HireAvailable'] = all_data['HireAvailable'].map({'Yes': 1, 'No': 0})
all_data['PublicHoliday'] = all_data['Public Holiday'].map({'Yes': 1, 'No': 0})
## Create a new column for weekend and assign 1 if the day is a weekend and 0 if it is not from date
all_data['Weekend'] = all_data['Date'].dt.dayofweek.map(lambda x: 1 if x > 4 else 0)
all_data['Season'] = all_data['Season'].map({'Spring': 0, 'Summer': 1, 'Autumn': 2, 'Winter': 3})
all_data['Month'] = all_data['Date'].dt.month
all_data['Year'] = all_data['Date'].dt.year
all_data['Day'] = all_data['Date'].dt.day

all_data.head()
all_data = all_data.dropna()

In [245]:
## split the data into training and testing
all_data = all_data.sample(frac=1).reset_index(drop=True)
test_data = all_data.sample(frac=0.2)
train_data = all_data.drop(test_data.index)
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)
print(test_data.size)

print(train_data.size)

35040
140160


In [246]:
x_train = [[train_data['PublicHoliday'][i], train_data['Weekend'][i], train_data['Month'][i], train_data['Hour'][i], train_data['Temp'][i], train_data['Windspeed'][i], train_data['Visibility'][i]] for i in range(len(train_data))]
y_train = [[train_data['Count'][i]] for i in range(len(train_data))]
x_test = [[test_data['PublicHoliday'][i], test_data['Weekend'][i], test_data['Month'][i], test_data['Hour'][i], test_data['Temp'][i], test_data['Windspeed'][i], test_data['Visibility'][i]] for i in range(len(test_data))]
y_test = [[test_data['Count'][i]] for i in range(len(test_data))]

print(x_train)
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)



[[1, 0, 12, 11, 3.1, 0.8, 804], [0, 1, 10, 10, 13.0, 0.5, 1685], [0, 1, 2, 16, -4.2, 3.5, 1974], [0, 1, 6, 7, 20.5, 0.8, 338], [1, 1, 9, 0, 17.2, 0.6, 2000], [0, 0, 1, 8, -15.1, 1.2, 2000], [0, 0, 6, 18, 28.1, 1.9, 959], [1, 0, 10, 4, 12.1, 1.2, 1834], [1, 0, 5, 9, 20.7, 1.1, 464], [1, 0, 5, 19, 17.6, 0.3, 331], [0, 0, 11, 10, 10.8, 1.8, 578], [0, 0, 5, 2, 12.5, 0.7, 1500], [0, 0, 10, 4, 9.7, 0.0, 1308], [0, 1, 3, 2, 0.9, 0.5, 1856], [0, 1, 8, 8, 30.2, 1.8, 1562], [0, 0, 2, 8, -3.3, 0.9, 1279], [0, 0, 4, 19, 7.6, 1.5, 1309], [0, 1, 5, 12, 21.9, 3.4, 1586], [0, 0, 10, 5, 8.5, 1.2, 1046], [0, 1, 2, 16, 0.3, 4.9, 2000], [0, 0, 12, 10, -3.7, 1.0, 541], [0, 1, 7, 18, 35.0, 2.9, 1953], [0, 0, 3, 4, 2.0, 0.8, 474], [0, 0, 6, 7, 19.4, 0.3, 1277], [0, 1, 9, 6, 18.4, 0.1, 752], [0, 0, 12, 10, -3.5, 1.2, 1996], [0, 0, 4, 17, 20.1, 1.9, 1088], [0, 1, 4, 14, 8.1, 4.7, 1914], [0, 0, 12, 21, -3.7, 0.0, 2000], [1, 1, 12, 9, 4.6, 0.9, 90], [0, 0, 5, 7, 12.3, 0.9, 1989], [0, 0, 5, 3, 16.6, 0.4, 402], [0

In [247]:
# Create Tensor datasets
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test)

# Create data loaders
batch_size = 8
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [248]:

# Initialize the neural network
model = NeuralNetwork()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)
print(len(train_loader))
# Training the neural network
epochs = 1000
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, targets in train_loader:
        model.train()
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ## gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        running_loss += loss.item()
    if epoch % 100 == 0:
      print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Testing the neural network
with torch.no_grad():
    model.eval()

    predictions = []
    for inputs in test_loader:
        outputs = model(inputs[0])
        predictions.append(outputs)
    predicted = torch.cat(predictions, dim=0)

print("Predicted values:", predicted)

876
Epoch [1/1000], Loss: 333210.6884
Epoch [101/1000], Loss: 179763.6658
Epoch [201/1000], Loss: 173747.0725
Epoch [301/1000], Loss: 168068.4687
Epoch [401/1000], Loss: 165459.8246
Epoch [501/1000], Loss: 162024.6979
Epoch [601/1000], Loss: 158565.1522
Epoch [701/1000], Loss: 153021.3071
Epoch [801/1000], Loss: 149238.4070
Epoch [901/1000], Loss: 148292.9312
Predicted values: tensor([[1830.6704],
        [1902.1731],
        [ 135.0453],
        ...,
        [1010.7719],
        [ 511.8423],
        [ 239.8940]])


In [249]:
print(len(train_loader))
from sklearn.metrics import r2_score
# Testing the neural network
with torch.no_grad():
    model.eval()

    predictions = []
    for inputs in test_loader:
        outputs = model(inputs[0])
        predictions.append(outputs)
    predicted = torch.cat(predictions, dim=0)
r2 = r2_score(y_test, predicted)

876


In [250]:
print("R2 Score:", r2)

R2 Score: 0.6268185207353183


In [26]:
with open("pg48320.txt") as f:
  text = f.read()

# find "adventures of sherlock holmes" index in text


# find "THE ADVENTURES OF SHERLOCK HOLMES" index in text

# slice the text from that index to the end
text2 = text[1133:567091]

len(text2)


565958

In [35]:
import re


# Define the regular expression pattern
pattern = r'\b(\w+)\s+and\s+(\w+)\b'

# Search for the pattern in the text
matches = re.findall(pattern, text)


# Filter matches where the last 3 characters match
filtered_matches = [(word1, word2) for word1, word2 in matches if word1[-3:] == word2[-3:]]
filtered_matches_all = []

# Print the filtered matches
for match in filtered_matches:
    filtered_matches_all.append((match[0] + " and " + match[1]).lower())

print(len(filtered_matches))
print(len(set(filtered_matches_all)))


49
45


In [36]:
filtered_matches_all

['reasoningandobserving',
 'smokingandlaughing',
 'thursdayandfriday',
 'inwardandoutward',
 'exactnessandastuteness',
 'depressingandsubduing',
 'detectedanddefeated',
 'selectionanddiscretion',
 'backwardandforward',
 'connivanceandassistance',
 'unlockingandthrowing',
 'lessandless',
 'mysteriesandimprobabilities',
 'bestandkeenest',
 'theoriesandfancies',
 'swiftlyandsilently',
 'examinedandretained',
 'totteringandshaking',
 'murderinganddriving',
 'claspingandunclasping',
 'januaryandfebruary',
 'hereandthere',
 'hereandthere',
 'coatandwaistcoat',
 'actionandreaction',
 'claspingsandunclaspings',
 'beatingandsplashing',
 'roundandround',
 'backwardandforward',
 'risingandputting',
 'louderandlouder',
 'quietlyandsecretly',
 'broadenedandbroadened',
 'dizzinessandsickness',
 'risingandbowing',
 'standingandlooking',
 'wooingandwedding',
 'puffingandblowing',
 'backwardandforward',
 'againandagain',
 'plankingandprobing',
 'bookandtook',
 'lowestandvilest',
 'oneandone',
 'backwar